In [1]:
import tensorflow as tf

In [2]:
import cv2

In [3]:
import graphviz
import pydot

In [4]:
import cv2
import os
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
import theano

import numpy as np





def norm(img,datashape=(13,55)):
    img = cv2.resize(img, (datashape[1], datashape[0]));
    #flag,img = cv2.threshold(img, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY)
    img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, \
                                   cv2.THRESH_BINARY, 5, 0.5)
    #
    # cv2.imshow("imgx",img);
    # cv2.waitKey(0)
    img = (img.astype(np.float32) / 255)
    shape = img.shape
    if len(shape) == 2:
        img -= img.mean()
        img = np.expand_dims(img, 2)
    else:
        img[:,:,1] -= img[:,:,1].mean()




    return img



def loadData(pathT,pathF,datashape=(13,55)):
    prepare_data =[]
    prepare_label = [];
    for parent,dirnames,filenames in os.walk(pathT):
        for filename in filenames:
            path = os.path.join(parent,filename)
            if path.endswith(".jpg") or path.endswith(".png") or path.endswith(".bmp"):
                img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
                img = norm(img,datashape)
                prepare_data.append(img);
                prepare_label.append([1,0]);

    for parent, dirnames, filenames in os.walk(pathF):
        for filename in filenames:
            path = os.path.join(parent, filename)
            if path.endswith(".jpg") or path.endswith(".png") or path.endswith(".bmp"):
                img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
                img = norm(img,datashape)

                prepare_data.append(img);
                prepare_label.append([0,1]);

    return np.array(prepare_data),np.array(prepare_label)




def arrangeData(data):
    prepare_data, prepare_label = data;
    rand = np.random.RandomState(321)
    shuffle = rand.permutation(len(prepare_data))
    shuffle = np.random.permutation(len(prepare_data));

    digits, labels = prepare_data[shuffle], prepare_label[shuffle]
    return digits,labels



def constructmodel(inputshape):
    model = Sequential()
    extract_conv1 = Convolution2D(8, 3, 3, border_mode='valid', input_shape=(inputshape))
    model.add(extract_conv1)
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Convolution2D(16, 2, 2, border_mode='valid'))
    # model.add(Convolution2D(64, 1, 1, border_mode='valid'))
    # model.add(Activation('relu'))
    # model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, init='normal'))
    model.add(Activation('relu'))
    model.add(Dropout(0.50))

    # model.add(Dense(32, init='normal'))
    # model.add(Activation('relu'))
    model.add(Dense(2, init='normal'))
    model.add(Activation('softmax'))

    return model

def train(pathT,pathF):
    model = constructmodel((13,55,1))
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=["accuracy"])

    data = loadData(pathT,pathF);

    training_data,training_label  = arrangeData(data)
    print (training_label.shape,training_data.shape)
    print (training_label)
    model.fit(training_data,training_label,nb_epoch=50,validation_split=0.1,show_accuracy=True)
    model.save("./judge1.h5")








#
#
#train("./training_T","./training_F")
#


Using TensorFlow backend.
WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'
WARNING (theano.tensor.blas): Failed to import scipy.linalg.blas, and Theano flag blas.ldflags is empty. Falling back on slower implementations for dot(matrix, vector), dot(vector, matrix) and dot(vector, vector) (cannot import name 'NUMPY_MKL')


In [5]:
model = constructmodel((13,55,1))

C:\Users\zhaoke1\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:78: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), padding="valid", input_shape=(13, 55, 1...)`
C:\Users\zhaoke1\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:84: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (2, 2), padding="valid")`
C:\Users\zhaoke1\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:89: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_initializer="normal")`
C:\Users\zhaoke1\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:95: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2, kernel_initializer="normal")`


In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 11, 53, 8)         80        
_________________________________________________________________
activation_1 (Activation)    (None, 11, 53, 8)         0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 26, 8)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 26, 8)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 25, 16)         528       
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               204928    
__________

In [7]:
from keras.utils.vis_utils import plot_model
from IPython.display import Image

plot_model(model, to_file="model.png", show_shapes=True)
Image('model.png')

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.

In [8]:
import pydot

In [9]:
dir(pydot)

['CLUSTER_ATTRIBUTES',
 'Cluster',
 'Common',
 'Dot',
 'EDGE_ATTRIBUTES',
 'Edge',
 'Error',
 'GRAPH_ATTRIBUTES',
 'Graph',
 'InvocationException',
 'NODE_ATTRIBUTES',
 'Node',
 'PY3',
 'Subgraph',
 '__author__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__license__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__version__',
 'copy',
 'division',
 'dot_keywords',
 'dot_parser',
 'frozendict',
 'graph_from_adjacency_matrix',
 'graph_from_dot_data',
 'graph_from_dot_file',
 'graph_from_edges',
 'graph_from_incidence_matrix',
 'id_re_alpha_nums',
 'id_re_alpha_nums_with_ports',
 'id_re_dbl_quoted',
 'id_re_html',
 'id_re_num',
 'id_re_with_port',
 'io',
 'needs_quotes',
 'os',
 'print_function',
 'quote_if_necessary',
 're',
 'str_type',
 'subprocess',
 'sys',
 'tempfile',
 'warnings']